# Part 1 - Run simulations for some general correlations and scalings

### First load some references

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

### Init Database etc.

In [ ]:
string ProjectName = "SFB1194_K65_Part1";

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();
myBatch // print queue information

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Setup Simulations

In [ ]:
double[] deltaS = new double[] {
    0.001, // millimeters
    //0.0005, 
    0.0001, 
    //0.00005, 
    0.00001, 
    //0.000005, 
    0.000001 }; // micro-meter
double[] V = new double[] {
  0.0,
  0.1, 
  //0.5, 
  1.0,
  //5.0, 
  10.0}; // Wall velocities in m/s
double[] pressureDiff = new double[] { 
  0.0,
  100.0,
  //10000.0,
  1000000.0,
  //100000000.0,
  10000000000.0 }; // imprinted pressure (one side +, other - p) in Pa

// always same
int Res = 10;
int DGdegree = 5;
double R = 0.1;

// set grid to be saved in database
GridFactory.myDb = myDb;

In [ ]:
var controls = new List<XNSE_Control>();

foreach(double delta in deltaS) {
   foreach(double V_wall in V) {
      foreach(double p in pressureDiff) {

         var C = new XNSE_Control();
         C.SetDGdegree(DGdegree);
         C.SetGrid(GridFactory.GenerateGrid(Res,delta, R));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("delta", delta));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Radius", R));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("V_Wall", V_wall));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("P_Diff", p));


         C.AddBoundaryValue("wall_walze", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
         C.AddBoundaryValue("wall_walze", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
         C.AddBoundaryValue("wall_substrat", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
         C.AddBoundaryValue("wall_substrat", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
         C.AddBoundaryValue("pressure_outlet_in", "Pressure", $"X => {p}", false);
         C.AddBoundaryValue("pressure_outlet_out", "Pressure", $"X => {-p}", false);


         C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

         C.PhysicalParameters.rho_A             = 1026.4; // not really relevant as this is steady-state
         C.PhysicalParameters.mu_A              = 0.0395;
         C.PhysicalParameters.IncludeConvection = false;
         C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
         
         C.SessionName = "J" + (20*Res*Res) + "_delta" + delta + "_V" + V_wall + "_P" + p;     
         controls.Add(C);
      }
   }
}

In [ ]:
Console.WriteLine(controls.Count);

### Run Simulations

Workaround so we need to deploy the rather large executables only once!

In [ ]:
string mngdir = "SFB1194_K65_Part1_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_managed";
DirectoryInfo ManagedOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, mngdir));
ManagedOverride.Create();
typeof(XNSE<XNSE_Control>).Assembly.DeployAt(ManagedOverride);
string RelManagedPath = "../" + mngdir + "/" + Path.GetFileName(typeof(XNSE<XNSE_Control>).Assembly.Location);

In [ ]:
string ntvdir = "SFB1194_K65_Part1_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_amd64";
DirectoryInfo NativeOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, ntvdir));
NativeOverride.Create();
MetaJobMgrIO.CopyDirectoryRec(ilPSP.Environment.NativeLibraryDir, NativeOverride.FullName, null);
myBatch.DeployRuntime = false; // we did this manually!

In [ ]:
foreach(var c in controls) {
    Job j = c.CreateJob();
    j.EnvironmentVars.Add(BoSSS.Foundation.IO.Utils.BOSSS_NATIVE_OVERRIDE, NativeOverride.FullName);
    j.EntryAssemblyRedirection = RelManagedPath;
    j.Activate(myBatch);
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(18000);